In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [4]:
# Load Data
data = pd.read_csv("D:/Projects/Churn_Modelling/data/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Preprocess the daa
## Drop irrelevant features
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Encode categorical feature
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
# Onehot Encoding 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [11]:
# Concat all columns with data
data = pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()                

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
## Save encoder and scaler
with open('D:/Projects/Churn_Modelling/models/label_encoder_genger.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('D:/Projects/Churn_Modelling/models/onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [13]:
# Divide data into Independent and Dependent feature
X = data.drop('Exited',axis=1)
y = data['Exited']

In [14]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [15]:
# Save scaler file
with open('D:/Projects/Churn_Modelling/models/scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implimentation

In [44]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [45]:
# Build ANN model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),    #1st Hidden Layer connected with input layer
    Dense(32,activation='relu'),                           # 2nd Hidden layer
    Dense(1,activation='sigmoid')                           # Output layer
]                              
)

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [48]:
## Compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [56]:
# Setup tensorboard for logs
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "D:/Projects/Churn_Modelling/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [57]:
# Setup EarlyStopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [58]:
# Training the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3342 - accuracy: 0.8639 - val_loss: 0.3496 - val_accuracy: 0.8565
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3284 - accuracy: 0.8636 - val_loss: 0.3507 - val_accuracy: 0.8600
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3274 - accuracy: 0.8643 - val_loss: 0.3457 - val_accuracy: 0.8545
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3258 - accuracy: 0.8662 - val_loss: 0.3567 - val_accuracy: 0.8535
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3236 - accuracy: 0.8668 - val_loss: 0.3453 - val_accuracy: 0.8610
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3262 - accuracy: 0.8662 - val_loss: 0.3418 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3202 - accuracy: 0.8684 - val_loss: 0.3489 - val_accuracy: 0.8595

In [59]:
model.save('D:/Projects/Churn_Modelling/models/model.h5')

In [60]:
# Load Tensorboard Extention
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [61]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 26456), started 0:23:31 ago. (Use '!kill 26456' to kill it.)